In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# import pybithumb
import numpy as np
import pandas as pd
from datetime import datetime
import os
# from scipy import stats
# from asq.initiators import query
import time
import matplotlib.pyplot as plt

gdrive_path = '/content/gdrive/My Drive/Colab Notebooks/'

In [4]:
def macd(df, short=12, long=26, signal=9):

    df['MACD'] = df['close'].ewm(span=short, min_periods=short-1, adjust=False).mean() - \
        df['close'].ewm(span=long, min_periods=long-1, adjust=False).mean()
    df['MACD_Signal'] = df['MACD'].ewm(span=signal, min_periods=signal-1, adjust=False).mean()
    df['MACD_OSC'] = df.MACD - df.MACD_Signal
    df['MACD_Zero'] = 0

    return


def max_abs_scaler(x):
    scaled_x = x / abs(x).max()
    return scaled_x


def profitage(Coin, short, long, signal, wait_tick=3, Date='2019-09-25', excel=0, get_fig=0):

    # try:
    #     df = pd.read_excel(dir + '%s %s ohlcv.xlsx' % (Date, Coin), index_col=0)
    #
    # except Exception as e:
    #     print('Error in loading ohlcv_data :', e)
    #     return 1.0, 1.0, 1.0
    # df = pybithumb.get_ohlcv(Coin, 'KRW', 'minute1')
    df = pd.read_excel(gdrive_path + './BestSet/Test_ohlc/%s' % Coin, index_col=0)

    macd(df, short=short, long=long, signal=signal)

    trade_state = [0] * len(df)
    for i in range(1, len(df)):
        if df['MACD_OSC'][i - 1] <= 0:
            if df['MACD_OSC'][i] > 0:
                trade_state[i] = 1

        if df['MACD_OSC'][i - 1] >= 0:
            if df['MACD_OSC'][i] < 0:
                trade_state[i] = 3

    for i in range(len(trade_state)):
        if trade_state[i] == 1.:  # 1 찾았을 때
            for j in range(i + 1, len(trade_state)):
                if trade_state[j] == 3.:  # 뒤에 2 존재한다면,
                    # print(i, j)
                    max_index = np.argmax(df['MACD_OSC'][i:j + 1].values)
                    if trade_state[i + max_index] != 1:
                        trade_state[i + max_index] = 2
                    # print(trade_state[i:j + 1])
                    # print(df['MACD_OSC'][i:j + 1].values)
                    # quit()
                    break

    df['trade_state'] = trade_state

    # plt.plot(df['MACD_OSC'].values)
    # plt.plot(df['MACD_Zero'].values, 'g')
    # span_list = list()
    # span_list2 = list()
    # for i in range(len(trade_state)):
    #     if trade_state[i] == 1.:
    #         span_list.append((i, i + 1))
    #     elif trade_state[i] == 2.:
    #         span_list2.append((i, i + 1))
    #
    # for i in range(len(span_list)):
    #     plt.axvspan(span_list[i][0], span_list[i][1], facecolor='c', alpha=0.7)
    # for i in range(len(span_list2)):
    #     plt.axvspan(span_list2[i][0], span_list2[i][1], facecolor='m', alpha=0.7)
    #
    # plt.show()
    # quit()

    # 매수 시점 = 급등 예상시, 매수가 = 이전 종가
    df['BuyPrice'] = np.where(df['trade_state'] == 1., df['close'], np.nan)
    # df['BuyPrice'] = df['BuyPrice'].apply(clearance)

    # 거래 수수료
    fee = 0.005

    # DatetimeIndex 를 지정해주기 번거로운 상황이기 때문에 틱을 기준으로 거래한다.
    # DatetimeIndex = df.axes[0]

    # ------------------- 상향 / 하향 매도 여부와 이익률 계산 -------------------#

    # high 가 SPP 를 건드리거나, low 가 SPM 을 건드리면 매도 체결 [ 매도 체결될 때까지 SPP 와 SPM 은 유지 !! ]
    length = len(df.index) - 1  # 데이터 갯수 = 1, m = 0  >> 데이터 갯수가 100 개면 m 번호는 99 까지 ( 1 - 100 )

    # 병합할 dataframe 초기화
    bprelay = pd.DataFrame(index=df.index, columns=['bprelay'])
    condition = pd.DataFrame(index=df.index, columns=["Condition"])
    Profits = pd.DataFrame(index=df.index, columns=["Profits"])
    # price_point = pd.DataFrame(index=np.arange(len(df)), columns=['Price_point'])

    Profits.Profits = 1.0
    Minus_Profits = 1.0

    # 오더라인과 매수가가 정해진 곳에서부터 일정시간까지 오더라인과 매수가를 만족할 때까지 대기  >> 일정시간,
    m = 0
    while m <= length:

        while True:  # bp 찾기
            if pd.notnull(df.iloc[m]['BuyPrice']):
                break
            m += 1
            if m > length:
                break

        if (m > length) or pd.isnull(df.iloc[m]['BuyPrice']):
            break
        bp = df.iloc[m]['BuyPrice']

        #       매수 등록 완료, 매수 체결 대기     #

        start_m = m
        while True:
            bprelay["bprelay"].iloc[m] = bp

            # 매수 체결 조건
            if df.iloc[m]['low'] <= bp: #and (df.iloc[m]['high'] != df.iloc[m]['low']):  # 조건을 만족할 경우 spp 생성
                # print(df['low'].iloc[m], '<=', bp, '?')

                condition.iloc[m] = '매수 체결'
                # bprelay["bprelay"].iloc[m] = bprelay["bprelay"].iloc[m - 1]

                m += 1
                break
            else:
                m += 1
                if m > length:
                    break
                if m - start_m >= wait_tick:
                    break

                condition.iloc[m] = '매수 대기'
                bprelay["bprelay"].iloc[m] = bprelay["bprelay"].iloc[m - 1]

    # df = pd.merge(df, condition, how='outer', left_index=True, right_index=True)
    # df = pd.merge(df, bprelay, how='outer', left_index=True, right_index=True)

    # if excel == 1:
    #     df.to_excel("./BackTest/%s BackTest %s.xlsx" % (Date, Coin))
    #     quit()

    #           지정 매도가 표시 완료           #

    #                               수익성 검사                                  #

    m = 0
    spanlist = []
    spanlist_limit = []
    spanlist_breakaway = []
    while m <= length:  # 초반 시작포인트 찾기

        while True: # SPP 와 SPM 찾긴
            if condition.iloc[m]['Condition'] == '매수 체결':
                # and type(df.iloc[m]['SPP']) != str:  # null 이 아니라는 건 오더라인과 매수가로 캡쳐했다는 거
                break
            m += 1
            if m > length: # 차트가 끊나는 경우, 만족하는 spp, spm 이 없는 경우
                break

        if (m > length) or pd.isnull(condition.iloc[m]['Condition']):
            break

        start_m = m

        #           매수 체결 지점 확인          #

        #       Predict value == 1 지점으로 부터 일정 시간 (10분) 지난 후 checking      #
        # while m - start_m <= over_tick:
        #     m += 1
        #     if m > length:
        #         break
        #     condition.iloc[m] = '매도 대기'
        #     bprelay["bprelay"].iloc[m] = bprelay["bprelay"].iloc[m - 1]
        # if m > length:
        #     break

        while True:

            #       고점 예측시 매도       #
            if df.iloc[m]['trade_state'] == 2. or df.iloc[m]['trade_state'] == 3.:
                break

            m += 1
            if m > length:
                break

            condition.iloc[m] = '매도 대기'
            bprelay["bprelay"].iloc[m] = bprelay["bprelay"].iloc[m - 1]

        if m > length:
            # print(condition.iloc[m - 1])
            break

        elif df.iloc[m]['trade_state'] == 2. or df.iloc[m]['trade_state'] == 3.:
            condition.iloc[m] = "매도 체결"
            Profits.iloc[m] = df.iloc[m]['close'] / bprelay["bprelay"].iloc[m - 1] - fee

            if float(Profits.iloc[m]) < 1:
                Minus_Profits *= float(Profits.iloc[m])
                try:
                    spanlist.append((start_m, m))
                    spanlist_breakaway.append((start_m, m))

                except Exception as e:
                    pass

            else:
                try:
                    spanlist.append((start_m, m))
                    spanlist_limit.append((start_m, m))

                except Exception as e:
                    pass

        # 체결시 재시작
        m += 1

    df = pd.merge(df, bprelay, how='outer', left_index=True, right_index=True)
    df = pd.merge(df, condition, how='outer', left_index=True, right_index=True)
    df = pd.merge(df, Profits, how='outer', left_index=True, right_index=True)
    # df = pd.merge(df, price_point, how='outer', left_index=True, right_index=True)
    df = df.reset_index(drop=True)

    if excel == 1:
        # df.to_excel("./BackTest/%s BackTest %s.xlsx" % (Date, Coin))
        df.to_excel("./%s BackTest %s.xlsx" % (Date, Coin))

    profits = Profits.cumprod()  # 해당 열까지의 누적 곱!

    if np.isnan(profits.iloc[-1].item()):
        return 1.0, 1.0, 1.0, 1.0, 1.0, 0.

    # [-1] 을 사용하려면 데이터가 존재해야 되는데 데이터 전체가 결측치인 경우가 존재한다.
    if len(profits) == 0:
        return 1.0, 1.0, 1.0, 1.0, 1.0, 0.

    elif float(profits.iloc[-1]) != 1.0 and get_fig == 1:

        # 거래 체결마다 subplot 1,2 저장
        plt.figure(figsize=(5, 5))
        plt.subplot(111)
        plt.plot(df[['MACD_OSC']], 'gold', label='oscillator', linewidth=1.0)
        plt.plot(df[['MACD_Zero']], 'g', label='zero', linewidth=1.0)
        plt.xticks(fontsize=10)
        plt.yticks(fontsize=10)
        plt.legend(loc='upper right', fontsize=5)
        plt.title('%.2f %.2f %.2f' % (float(profits.iloc[-1]), float(profits.iloc[-1]) / Minus_Profits, Minus_Profits))

        for trade_num in range(len(spanlist_limit)):
            plt.axvspan(spanlist_limit[trade_num][0], spanlist_limit[trade_num][1], facecolor='c', alpha=0.5)

        for trade_num in range(len(spanlist_breakaway)):
            plt.axvspan(spanlist_breakaway[trade_num][0], spanlist_breakaway[trade_num][1], facecolor='m', alpha=0.5)

        # plot 저장 & 닫기
        # plt.show()
        plt.savefig("./Figure_pred/%s_%s_%s/%s %s.png" % (short, long, signal, Date, Coin), dpi=300)
        plt.close()

    if profits.values[-1] != 1.:
      profits_sum = 0
      for i in range(len(Profits)):
          if Profits.values[i] != 1.:
              profits_sum += Profits.iloc[i]
      profits_avg = profits_sum / sum(Profits.values != 1.)

    else:
        profits_avg = [1.]
    # print(Profits.values.min())
    # quit()
    # print(Profits.values != 1.)
    # print(profits_avg)
    # quit()
    std = np.std(max_abs_scaler(df[df['trade_state'] == 2.]['MACD_OSC']))


    return float(profits.iloc[-1]), float(profits.iloc[-1]) / Minus_Profits, Minus_Profits, profits_avg[0], Profits.values.min(), std


if __name__=="__main__":

    # Coinlist = pybithumb.get_tickers()
    # Fluclist = []
    # while True:
    #     try:
    #         for Coin in Coinlist:
    #             tickerinfo = pybithumb.PublicApi.ticker(Coin)
    #             data = tickerinfo['data']
    #             fluctate = data['fluctate_rate_24H']
    #             Fluclist.append(fluctate)
    #             time.sleep(1 / 90)
    #         break
    #
    #     except Exception as e:
    #         Fluclist.append(None)
    #         print('Error in making Topcoin :', e)
    #
    # Fluclist = list(map(float, Fluclist))
    # series = pd.Series(Fluclist, Coinlist)
    # series = series.sort_values(ascending=False)
    #
    # series = series[:10]
    # TopCoin = list(series.index)
    # print(TopCoin)

    # print(Date)
    # quit()
    #     # Date = '2020-02-27'

    # ohlcv_list = ['2019-10-05 LAMB ohlcv.xlsx']

    # for file in ohlcv_list:
    #     Coin = file.split()[1].split('.')[0]
    #     Date = file.split()[0]
    Date = str(datetime.now()).split()[0]
    excel_list = os.listdir(gdrive_path + './BestSet/Test_ohlc/')
    total_df = pd.DataFrame(columns=['short', 'long', 'signal', 'total_profit_avg', 'plus_profit_avg', 'minus_profit_avg', 'avg_profit_avg', 'min_profit_avg', 'std_avg'])

    for short in range(10, 26, 5):
        for long in range(short + 3, short + 100, 5):
            for signal in range(5, 50, 3):

                # for Coin in TopCoin:
                #     print(short, long, signal)
                #     print(profitage(Coin, short, long, signal, Date=Date))

                #       Make folder      #
                # try:
                #     os.mkdir("./Figure_pred/%s_%s_%s/" % (short, long, signal))
                #
                # except Exception as e:
                #     print(e)
                # short, long, signal = 20, 33, 5

                total_profit = 0
                plus_profit = 0
                minus_profit = 0
                avg_profit = 0
                min_profit = 0
                std = 0
                for Coin in excel_list:
                    start = time.time()
                    result = profitage(Coin, short, long, signal, Date=Date, get_fig=0)
                    # quit()
                    total_profit += result[0]
                    plus_profit += result[1]
                    minus_profit += result[2]
                    avg_profit += result[3]
                    min_profit += result[4]
                    std += result[5]
                total_profit_avg = total_profit / len(excel_list)
                plus_profit_avg = plus_profit / len(excel_list)
                minus_profit_avg = minus_profit / len(excel_list)
                avg_profit_avg = avg_profit / len(excel_list)
                min_profit_avg = min_profit / len(excel_list)
                std_avg = std / len(excel_list)
                print(short, long, signal, total_profit_avg, plus_profit_avg, minus_profit_avg, avg_profit_avg, min_profit_avg, std_avg, '%.3f second' % (time.time() - start))

                result_df = pd.DataFrame(data=[[short, long, signal, total_profit_avg, plus_profit_avg, minus_profit_avg, avg_profit_avg, min_profit_avg, std_avg]],
                                         columns=['short', 'long', 'signal', 'total_profit_avg', 'plus_profit_avg', 'minus_profit_avg', 'avg_profit_avg', 'min_profit_avg', 'std_avg'])
                total_df = total_df.append(result_df)
                                
        total_df.to_excel(gdrive_path + './BestSet/total_df %s.xlsx' % short)
        # break

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:231: FutureWarning: `item` has been deprecated and will be removed in a future version


10 13 5 1.370205271233217 2.5988440223869875 0.5610934672085646 1.0059527109145274 0.891301330141847 0.20842118307818053 1.431 second
10 13 8 1.455123672683265 2.538416585232995 0.5928262343809958 1.0077411626652597 0.8806630588074176 0.21523048851213633 0.835 second
10 13 11 1.6294886158228068 2.4960618756856547 0.6761842374271682 1.0143192446160902 0.9167027998799255 0.21467480313208215 0.859 second
10 13 14 1.641336732200217 2.3607857314030474 0.7081019871064502 1.0146427015388002 0.9111540981697892 0.21664472460928197 0.804 second
10 13 17 1.663165621403067 2.3054243085468635 0.7380237946100646 1.0177452377104292 0.9134193591490813 0.21836116405463774 0.792 second
10 13 20 1.6126515083654116 2.205558908653059 0.7500846132075937 1.018172728793226 0.913557881928241 0.22136926126683748 0.781 second
10 13 23 1.6718664102544742 2.1891550058362026 0.7823471921054898 1.0204797298005057 0.92690444457953 0.22334114324811258 0.818 second
10 13 26 1.6072346234981336 2.1659211346903424 0.76172